In [5]:
import pandas as pd
import numpy as np
import nltk
import nltk.sentiment.util
import matplotlib.pyplot as plt
import pprint
import re
import gensim
import gensim.corpora as corpora
import os
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
from textblob import TextBlob

In [6]:
df = pd.read_csv("../../amazon_reviews_us_Grocery_v1_00.tsv", sep='\t', quoting=3)
df

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,42521656,R26MV8D0KG6QI6,B000SAQCWC,159713740,"The Cravings Place Chocolate Chunk Cookie Mix,...",Grocery,5,0,0,N,Y,Using these for years - love them.,"As a family allergic to wheat, dairy, eggs, nu...",2015-08-31
1,US,12049833,R1OF8GP57AQ1A0,B00509LVIQ,138680402,"Mauna Loa Macadamias, 11 Ounce Packages",Grocery,5,0,0,N,Y,Wonderful,"My favorite nut. Creamy, crunchy, salty, and ...",2015-08-31
2,US,107642,R3VDC1QB6MC4ZZ,B00KHXESLC,252021703,Organic Matcha Green Tea Powder - 100% Pure Ma...,Grocery,5,0,0,N,N,Five Stars,This green tea tastes so good! My girlfriend l...,2015-08-31
3,US,6042304,R12FA3DCF8F9ER,B000F8JIIC,752728342,15oz Raspberry Lyons Designer Dessert Syrup Sauce,Grocery,5,0,0,N,Y,Five Stars,I love Melissa's brand but this is a great sec...,2015-08-31
4,US,18123821,RTWHVNV6X4CNJ,B004ZWR9RQ,552138758,"Stride Spark Kinetic Fruit Sugar Free Gum, 14-...",Grocery,5,0,0,N,Y,Five Stars,good,2015-08-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2402453,US,50385921,RZVFIWM72OSA0,B00004TBB0,738532654,Celestial Seasonings - Herbal Tea Caffeine Fre...,Grocery,4,0,4,N,N,Different,The Amor Belhom Duo are likely the only musici...,2000-12-05
2402454,US,50377094,R35K50CF47311S,B00000K1X9,161697639,479 NATL SHOW HRSE BAY PINTO 6,Grocery,5,0,0,N,N,This Horse Is Gorgourges!,Being a Breyer collecter almost all of ...,1999-12-29
2402455,US,50740327,R1N80O7XL5ODNC,B00000K1X9,161697639,479 NATL SHOW HRSE BAY PINTO 6,Grocery,5,5,5,N,N,"brilliant, realistic model!",This model is beautiful. each model is hand p...,1999-11-21
2402456,US,51700314,R1E9RSEX4R6TIR,B00000K1X9,161697639,479 NATL SHOW HRSE BAY PINTO 6,Grocery,5,1,1,N,N,don't have it yet..but will soon,As a person who has collected Breyers all my l...,1999-11-17


In [7]:
df = df.dropna()

In [ ]:
polarity = []
subjectivity = []
for line in df['review_body']:
    line = line.replace('<br />', '')
    blob = TextBlob(line)
    pol = []
    sub = []
    for sentence in blob.sentences:
        pol.append(sentence.sentiment.polarity)
        sub.append(sentence.sentiment.subjectivity)
    polarity.append(np.array(pol).mean())
    subjectivity.append(np.array(sub).mean())
df['polarity'] = polarity
df['subjectivity'] = subjectivity

In [ ]:
df.loc[(df['polarity'] < 0) & (df['subjectivity'] < 0.1)]['review_body']

In [44]:
df.to_csv('../../amazon_reviews_us_Grocery_v1_00_sentiment.csv',index=False)

In [17]:
df.query('review_body.str.contains("diarrhea", na=False)', engine='python')

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
847,US,33974316,R1WM66IPQMCCUP,B000FKQDAQ,431762188,Life Savers 5 Flavors Sugarfree Hard Candy Bag...,Grocery,3,15,18,N,Y,"Delicious, but causes severe stomach pain and ...","Please read! These taste fantastic, however t...",2015-08-31
4295,US,813023,R1MC797GCYRVC,B007G92C6I,639406517,Youthberry White Tea by Teavana,Grocery,1,2,3,N,N,Bought a glass of this at the Teavana store. ...,Bought a glass of this at the Teavana store......,2015-08-30
8565,US,51402655,R3N8331434IVOG,B00IYVK3R2,169375071,Atkinsons Chick O Sticks Sugar Free 1lb Bag,Grocery,1,0,0,N,Y,CONTAINS INTESTINE-DISTURBING SUGAR ALCOHOL,These are sweetened with Splenda AND a sugar a...,2015-08-29
16240,US,11628162,R2UCFLANW09L4M,B000EVOSE4,990977895,"Haribo Gummi Candy Gold-Bears,",Grocery,4,9,14,N,Y,If You're Looking For Diarrhea Bears Get the S...,Wanted to stock the office with diarrhea bears...,2015-08-26
17077,US,13373792,R16FT1THCAPTC1,B00C584ALK,233975251,"Bai Cocofusions, Antioxidant Infused Drinks",Grocery,1,121,144,N,Y,I love it - my gut hates it!,I loved this product and drank it every day fo...,2015-08-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2394635,US,21578951,RWUKRXWR1GJ2Z,B00032G1S0,753469671,"Tuscan Dairy Whole Vitamin D Milk, Gallon, 128 oz",Grocery,5,58,64,N,N,A former Tuscaneer...,"My name is Zach, a former employee of Tuscan D...",2006-08-27
2395398,US,43293948,R2NJ2RQQQFXHJJ,B00032G1S0,753469671,"Tuscan Dairy Whole Vitamin D Milk, Gallon, 128 oz",Grocery,5,12,14,N,N,Protector of Mankind,Utterly moov-ing minute globules of fat suspen...,2006-08-11
2396546,US,11411331,R1EN6U6N06BYJC,B00032G1S0,753469671,"Tuscan Dairy Whole Vitamin D Milk, Gallon, 128 oz",Grocery,5,15,19,N,N,Lactose intolerance,I recently found out that my girlfriend is lac...,2006-07-21
2396561,US,10359874,R15QTN93IG51L2,B00032G1S0,753469671,"Tuscan Dairy Whole Vitamin D Milk, Gallon, 128 oz",Grocery,5,14,17,N,N,This milk is good,I was expecting this milk to be a normal gallo...,2006-07-21
